## Food - Nutrients            
            
Mounted knowledge base data sources: **source Food and nutrients**, see notebook **"Data source validation "** for details            
            
            
- Merge food data from foundation, SR legacy and Survey sources            
- Removing conflicting data

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import random
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
# from codes.utils import name_similar

In [2]:
food_found = pd.read_csv('../data/Foundation_food.csv')

### Read the data source

In [2]:
print('View data source: foundation foods ----------------------')
food_found = pd.read_csv('../data/Foundation_food.csv') # 源文件 food.csv
food_nutrition_found = pd.read_csv('../data/Foundation_food_nutrition.csv') # 源文件 food_nutrition.csv
print(f"food{food_found.shape}, food_nurtrition{food_nutrition_found.shape}")

print('\nView data source: SR Legacy -----------------------------')
food_sr = pd.read_csv('../data/SR_food.csv') # 源文件 food.csv
food_nutrition_sr = pd.read_csv('../data/SR_food_nutrient.csv') # 源文件 food_nutrition.csv
print(f"food{food_sr.shape}, food_nurtrition{food_nutrition_sr.shape}")


print('\nView data source: survey --------------------------------')
food_survey = pd.read_csv('../data/Survey_food.csv')
food_nutrition_survey = pd.read_csv('../data/Survey_food_nutrient.csv')
print(f"food{food_survey.shape}, food_nurtrition{food_nutrition_survey.shape}")


print('View data source: supporting data -----------------------')
nutrition = pd.read_csv('../data/nutrient.csv') # nutrient.csv
print(f"nurtrition {nutrition.shape}")

View data source: foundation foods ----------------------
food(43446, 5), food_nurtrition(122720, 11)

View data source: SR Legacy -----------------------------


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


food(7793, 5), food_nurtrition(644125, 11)

View data source: survey --------------------------------
food(7083, 5), food_nurtrition(460395, 11)
View data source: supporting data -----------------------
nurtrition (474, 5)


In [3]:

# food_nutrition_found

### Food Matching Nutrients & Homologous De-duplication            
            
1. food matching nutrients: food association food_nurtrition association nurtrition            
            
2. same-source de-duplication: there is a lot of duplication of nutrients for each food, same data source (sr, foundation, survey) with data_type under:            
- Only the most recent record is retained            
- If there are still duplicates in the most recent record, keep the median

In [4]:
def merge_food_nurtrition_info(food, food_nur, nur, df_name, drop = True):
    temp = pd.merge(food, food_nur, on='fdc_id',how='right')
    final_df = pd.merge(temp, nur,left_on='nutrient_id',right_on='id',how='left')[['fdc_id', 'data_type', 'publication_date','description','name','amount']]
    final_df['description'] = final_df['description'].apply(lambda x: str(x).lower())
    final_df['name'] = final_df['name'].apply(lambda x: str(x).lower())
    
    if drop:
        print(f'----------------Intermediate table based on {df_name}: \n 1. After food and nutrient association shape = {final_df.shape}')
        drop_duli = drop_duplicate_food(final_df)
        drop_duli['source'] = df_name
        print(f"2. After de-duplication based on the same data source and data_type shape = {drop_duli.shape}")
        display(drop_duli.head())
        return drop_duli
    else:
        return final_df

def drop_duplicate_food(food_data):
    # There is a lot of duplication of nutrients for each food and they correspond to different fdc_id (fdc_id is randomly assigned when a new version of the food appears)
    # 1. Only the most up-to-date nutrient records at the time of retention
    keys = food_data[['fdc_id', 'description', 'name', 'data_type', 'publication_date']]
    latest = keys.groupby(['description', 'name', 'data_type'], as_index = False)['publication_date'].max()
    latest_data = latest.merge(food_data, on = ['description', 'name', 'data_type', 'publication_date'], how='left')

    # 2. If multiple records exist for the latest time, the median is retained
    result = latest_data.groupby(['description', 'data_type', 'name']).agg({
        'fdc_id':['count'], 
        'amount':['median', 'min', 'max', 'var']}).reset_index()
    result.columns = ['description', 'data_type', 'name', 'fdc_id_count', 'median', 'min', 'max', 'var']
    result = result.sort_values('fdc_id_count', ascending = False)
    dupli = result[result['fdc_id_count']>1].reset_index().drop(columns=['index'])
    print(f'Duplicate and conflicting records under the same data_source {len(dupli)}')
    return result

def show_sample(food, food_nur, nur, summarised, k=0):
    found_dupli = summarised[summarised['fdc_id_count']>k].reset_index().drop(columns=['index'])
    found_dupli_demo = found_dupli.groupby(['description', 'name'], as_index = False)['var'].agg(['count', 'max'])
    found_dupli_demo = found_dupli_demo[found_dupli_demo['count']>1].sort_values(['description', 'max'], ascending = False).reset_index()[0:5] # 波动最大的前五

    # Get the full amount of data before de-duplication
    found_full = merge_food_nurtrition_info(food, food_nur, nur, '-', False) # base on foundation
    found_demo = found_full.merge(found_dupli_demo[['description', 'name']], on = ['description', 'name'], how = 'right')
    found_type = list(set(found_demo['data_type']))

    fig = plt.figure(figsize=(16,5), dpi = 60)
    for i in range(len(found_type)):
        ax1 = plt.subplot(121+i)
        sns.boxplot(x='name', y='amount', data=found_demo[found_demo['data_type']==found_type[i]])


In [10]:
food_survey.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,1097510,survey_fndds_food,"Milk, human",NaN,2020-10-30
1,1097511,survey_fndds_food,"Milk, NFS",NaN,2020-10-30
2,1097512,survey_fndds_food,"Milk, whole",NaN,2020-10-30
3,1097513,survey_fndds_food,"Milk, low sodium, whole",NaN,2020-10-30
4,1097514,survey_fndds_food,"Milk, calcium fortified, whole",NaN,2020-10-30


In [5]:
food_nurtrition_survey_info = merge_food_nurtrition_info(food_survey, food_nutrition_survey, nutrition, 'survey') # base on survey

----------------Intermediate table based on survey: 
 1. After food and nutrient association shape = (460395, 6)
Duplicate and conflicting records under the same data_source 0
2. After de-duplication based on the same data source and data_type shape = (460395, 9)


,description,data_type,name,fdc_id_count,median,min,max,var,source
0,100 grand bar,survey_fndds_food,"alcohol, ethyl",1,0.0,0.0,0.0,NaN,survey
306855,"pepper, raw, nfs",survey_fndds_food,vitamin b-12,1,0.0,0.0,0.0,NaN,survey
306937,"pepper, sweet, green, raw",survey_fndds_food,"choline, total",1,5.5,5.5,5.5,NaN,survey
306936,"pepper, sweet, green, raw",survey_fndds_food,cholesterol,1,0.0,0.0,0.0,NaN,survey
306935,"pepper, sweet, green, raw",survey_fndds_food,"carotene, beta",1,208.0,208.0,208.0,NaN,survey


In [6]:
food_nurtrition_found_info = merge_food_nurtrition_info(food_found, food_nutrition_found, nutrition, 'foundation') # base on foundation

----------------Intermediate table based on foundation: 
 1. After food and nutrient association shape = (122720, 6)
Duplicate and conflicting records under the same data_source 5956
2. After de-duplication based on the same data source and data_type shape = (66296, 9)


,description,data_type,name,fdc_id_count,median,min,max,var,source
977,american cheese,sub_sample_food,glucose,59,0.00,0.00,0.22,0.002603,foundation
980,american cheese,sub_sample_food,lactose,59,3.45,0.72,6.34,1.465061,foundation
1033,american cheese,sub_sample_food,sucrose,59,0.00,0.00,0.00,0.000000,foundation
982,american cheese,sub_sample_food,maltose,59,0.00,0.00,0.00,0.000000,foundation
975,american cheese,sub_sample_food,fructose,59,0.00,0.00,0.00,0.000000,foundation


In [13]:
# Case Studies
found_dupli = food_nurtrition_found_info[food_nurtrition_found_info['fdc_id_count']>1].reset_index().drop(columns=['index'])
found_dupli_demo = found_dupli.groupby(['description', 'name'], as_index = False)['var'].agg(['count', 'max'])
found_dupli_demo = found_dupli_demo[found_dupli_demo['count']>1].sort_values(['description', 'max'], ascending = False).reset_index()[0:5] # Top 5 most volatile

# Get the full amount of data before de-duplication
found_full = merge_food_nurtrition_info(food_found, food_nutrition_found, nutrition, 'foundation', False) # base on foundation
found_demo = found_full.merge(found_dupli_demo[['description', 'name']], on = ['description', 'name'], how = 'right')
found_type = list(set(found_demo['data_type']))

fig = plt.figure(figsize=(18,5), dpi = 60)
ax1 = plt.subplot(121)
sns.boxplot(x='name', y='amount', data=found_demo[found_demo['data_type']==found_type[0]])
ax1 = plt.subplot(122)
sns.boxplot(x='name', y='amount', data=found_demo[found_demo['data_type']==found_type[1]])

<Figure size 1080x300 with 2 Axes>

In [7]:
food_nurtrition_SR_info = merge_food_nurtrition_info(food_sr, food_nutrition_sr, nutrition, 'SR legacy') # base on legacy

----------------Intermediate table based on SR legacy: 
 1. After food and nutrient association shape = (644125, 6)
Duplicate and conflicting records under the same data_source 7793
2. After de-duplication based on the same data source and data_type shape = (636332, 9)


,description,data_type,name,fdc_id_count,median,min,max,var,source
362071,"lamb, new zealand, imported, square-cut should...",sr_legacy_food,energy,2,661.5,255.0,1068.0,330484.5,SR legacy
331649,"ice creams, vanilla, fat free",sr_legacy_food,energy,2,358.0,138.0,578.0,96800.0,SR legacy
173055,"cabbage, mustard, salted",sr_legacy_food,energy,2,72.0,28.0,116.0,3872.0,SR legacy
507781,"restaurant, chinese, sweet and sour pork",sr_legacy_food,energy,2,699.5,270.0,1129.0,368940.5,SR legacy
410120,"nuts, walnuts, glazed",sr_legacy_food,energy,2,1296.0,500.0,2092.0,1267232.0,SR legacy


In [15]:
# Case Studies
sr_dupli = food_nurtrition_SR_info[food_nurtrition_SR_info['fdc_id_count']>1].reset_index().drop(columns=['index'])[0:1]

# Get the full amount of data before de-duplication
sr_full = merge_food_nurtrition_info(food_sr, food_nutrition_sr, nutrition, 'foundation', False) # base on foundation
found_demo = sr_full.merge(sr_dupli[['description', 'name']], on = ['description', 'name'], how = 'right')
found_type = list(set(found_demo['data_type']))

fig = plt.figure(figsize=(18,5), dpi = 60)
ax1 = plt.subplot(121)
sns.boxplot(x='name', y='amount', data=found_demo[found_demo['data_type']==found_type[0]])

<Figure size 1080x300 with 1 Axes>

### Three source merger            


In [16]:
# Consolidated into a summary table
food_nurtrition = pd.concat([food_nurtrition_found_info, food_nurtrition_survey_info, food_nurtrition_SR_info])
food_nurtrition['source_type'] = food_nurtrition['source']+': '+food_nurtrition['data_type']
food_nurtrition['source_type'] = food_nurtrition['source_type'].apply(lambda x: x.lower())
print(f'原始的总表 shape: {food_nurtrition.shape}')
display(food_nurtrition.head())

原始的总表 shape: (1163023, 10)


,description,data_type,name,fdc_id_count,median,min,max,var,source,source_type
977,american cheese,sub_sample_food,glucose,59,0.00,0.00,0.22,0.002603,foundation,foundation: sub_sample_food
980,american cheese,sub_sample_food,lactose,59,3.45,0.72,6.34,1.465061,foundation,foundation: sub_sample_food
1033,american cheese,sub_sample_food,sucrose,59,0.00,0.00,0.00,0.000000,foundation,foundation: sub_sample_food
982,american cheese,sub_sample_food,maltose,59,0.00,0.00,0.00,0.000000,foundation,foundation: sub_sample_food
975,american cheese,sub_sample_food,fructose,59,0.00,0.00,0.00,0.000000,foundation,foundation: sub_sample_food


### Heterogeneous source de-duplication            
            
There are still a large number of duplicates (4172) of the same food under different data sources and different datatypes, further de-duplication:            
            
- Retained according to priority in "source_credibility_score.csv", priority=1 highest priority

In [17]:
# Repetition
check_duplicate = food_nurtrition.groupby(["description", "name"])['median'].agg(["max", "min"]).reset_index()
check_duplicate = check_duplicate[check_duplicate['max']>check_duplicate['min']]
print(f'直接拼接上述去重数据源，会导致相同食物相同营养物质存在多个营养物质含量，重复项{len(check_duplicate)}个')

dupli_sources = check_duplicate.merge(food_nurtrition[['description', 'name', 'source_type']], on = ['description', 'name'], how='left')
dupli_sources = dupli_sources.groupby(["description", "name"])['source_type'].agg('unique').reset_index()
dupli_sources['dupli_s'] = dupli_sources['source_type'].apply(lambda x: str(sorted(x)))
dupli_sources.groupby('dupli_s').agg('count')

直接拼接上述去重数据源，会导致相同食物相同营养物质存在多个营养物质含量，重复项4172个


,description,name,source_type
dupli_s,,,
"['foundation: foundation_food', 'foundation: sub_sample_food']",933,933,933
"['foundation: foundation_food', 'sr legacy: sr_legacy_food', 'survey: survey_fndds_food']",131,131,131
"['foundation: foundation_food', 'sr legacy: sr_legacy_food']",2231,2231,2231
"['foundation: foundation_food', 'survey: survey_fndds_food']",2,2,2
"['foundation: sub_sample_food', 'survey: survey_fndds_food']",96,96,96
"['sr legacy: sr_legacy_food', 'survey: survey_fndds_food']",779,779,779


In [22]:
# Priority
priority = pd.read_csv('../data/来源可信度_score.csv')
priority['source_type'] = priority['source']+': ' + priority['data_type']
priority

,source,data_type,comment,priority,source_type
0,foundation,foundation_food,集合多个样本,1,foundation: foundation_food
1,foundation,sub_sample_food,NaN,5,foundation: sub_sample_food
2,foundation,market_acquisition,为了化学实验采集来的,3,foundation: market_acquisition
3,foundation,agricultural_acquisition,直接从原产地采集来的,3,foundation: agricultural_acquisition
4,foundation,sample_food,NaN,4,foundation: sample_food
5,sr legacy,sr_legacy_food,USDA National Nutrient Database(背书),2,sr legacy: sr_legacy_food
6,survey,survey_fndds_food,食物营养摄入量（NHANES），有经过计算融合了brading和sr leagacy data,6,survey: survey_fndds_food


In [23]:
# De-weighting according to priority
unique_food_nurtrition = food_nurtrition.merge(priority[['source_type', 'priority']], on='source_type', how='left')
unique_food_nurtrition['keep_prio'] = unique_food_nurtrition.groupby(['description', 'name']).pipe(lambda x: x.priority.transform('min'))
unique_food_nurtrition = unique_food_nurtrition[unique_food_nurtrition['keep_prio']==unique_food_nurtrition['priority']]
print('最终去重后的数据 shape：', unique_food_nurtrition.shape)
print('涉及食物种类：', len(set(unique_food_nurtrition['description'])))

最终去重后的数据 shape： (1151062, 12)
涉及食物种类： 21675


In [24]:
food_features = unique_food_nurtrition.pivot(index = 'description', columns = 'name', values = 'median').reset_index()
print('查看原始的中间表信息-------------------------------------')
print(f"食物 x 营养物质（用于构建特征）：{food_features.shape}")
food_features.head()

查看原始的中间表信息-------------------------------------
食物 x 营养物质（用于构建特征）：(21675, 245)


name,description,"ergosta-5,7-dienol","ergosta-7,22-dienol",10-formyl folic acid (10hcofa),25-hydroxycholecalciferol,5-formyltetrahydrofolic acid (5-hcoh4,5-methyl tetrahydrofolate (5-mthf),alanine,"alcohol, ethyl",arginine,...,vitamin d3 (cholecalciferol),vitamin d4,vitamin e (alpha-tocopherol),"vitamin e, added",vitamin k (dihydrophylloquinone),vitamin k (menaquinone-4),vitamin k (phylloquinone),water,zeaxanthin,"zinc, zn"
0,egg whites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
1,"egg whites, dried",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.49
2,egg yolk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.96
3,"egg yolks, dried",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.26
4,whole eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.21


## food-compounds            
            
Mounted knowledge base data source: **source Food & Nutrients**, see notebook **"Data Source Validation "** for details            
            
            
- Simple pre-processing of crawled data            
- Food compound content, scaled to between 0.01 and 1 in compounds (if a food f has the highest compound a and b content of all foods, then fa=1 & fb=1)            
            
            
### Read the data source

In [14]:
web_food = pd.read_excel('/home/mw/input/sr_food_nur5672/FoodDB爬取数据.xlsx')#('/home/mw/input/web_food3967/初始完整版.xlsx')
display(web_food.head())

### Pre-processing

In [15]:
web_food = web_food.drop('id', axis=1)
pattern = r"[a-z A-Z]"
cond_a = web_food['平均值'].astype(str).str.contains(pattern, regex=True)
cond_b = web_food['最小值'].astype(str).str.contains(pattern, regex=True)
cond_c = web_food['最大值'].astype(str).str.contains(pattern, regex=True)
cond = zip(cond_a.to_list(), cond_b.to_list(), cond_c.to_list())
cond = map(lambda x: x[0] or x[1] or x[2], cond)
cond = pd.Series(list(cond))
web_food = web_food[~cond]
# Find a list of unique compounds
comlist_web = web_food['化合物名称'].tolist()
comlist_web = set(comlist_web)

In [16]:
#Formatting & Converting names to lower case
food_compound = web_food[["食物名称","化合物名称", "平均值", "计量单位"]]
food_compound.columns = ["food_name", "compound_name", "standard_content", "orig_unit"]
food_compound = food_compound.reset_index()

food_compound['food_lower'] = food_compound['food_name'].apply(lambda x: x.lower())
food_compound['cmp_lower'] = food_compound['compound_name'].apply(lambda x: x.lower())

# Data scaling
scaler1=preprocessing.MinMaxScaler()
food_compound = food_compound.join(food_compound.groupby(['cmp_lower'])[['standard_content']].apply(lambda x: pd.DataFrame(scaler1.fit_transform(x), index=x.index, columns=['scaler'])))
food_compound['scaler'] = 0.01 + food_compound['scaler']*0.99

unique_food_compound = list(set(food_compound['cmp_lower']))
unique_food = list(set(food_compound['food_lower']))

display(food_compound.head())
print(f'食物{len(unique_food)}：共涉及化合物{len(unique_food_compound)}种')

## Nutrients - Food - Compounds            
            
Nutrient-related food names do not match compound-related food names. To establish the correspondence, the relationship tables food_matchindisease_nutrition.csv, food_rela2.csv are constructed manually + in code            
            
### Read the relationship table            
            
food_disease: from food compounds            
long: from food nutrient

In [98]:
food_name_match_manul = pd.read_csv('../data/food_matchindisease_nutrition.csv').drop_duplicates()
food_name_match_manul = food_name_match_manul[
    food_name_match_manul['food_disease'].isin(food_compound['food_lower']) &
    food_name_match_manul['long'].isin(food_features['description']) ]
food_name_match_manul = food_name_match_manul[~food_name_match_manul['food_disease'].isin(['dill', 'water'])]

food_name_2 = pd.read_csv('../data/food_rela2.csv')
food_name_match = pd.concat([food_name_match_manul, food_name_2])
print(f"将 营养物质_食物 {len(set(food_name_match['long']))} 与 化合物_食物 {len(set(food_name_match['food_disease']))} 建立关联")
food_name_match.head()

In [69]:
# 运行下述注释代码，可生成 food_name_2 补充关联表的源
# enhence_match = pd.DataFrame()
# left_long = set(food_features['description'])-set(food_name_match_manul['long'])
# left_nur = set(food_compound['food_lower'])-set(food_name_match_manul['food_disease'])
# for nur in left_nur:
#     sub_df = pd.DataFrame({'food_disease': nur, 'long': list(left_long)})
#     sub_df['score'] = sub_df['long'].apply(lambda x: name_similar(x, nur))
#     sub_df = sub_df.sort_values('score', ascending=False).reset_index(drop=True)[0:10]
#     enhence_match = pd.concat([enhence_match, sub_df[sub_df['score']>0.6]])
# enhence_match.to_csv('/home/mw/temp/temp_match.csv', index=False)

,food_disease,long
0,alfalfa,"alfalfa sprouts, raw"
1,almond,"almonds, nfs"
2,apple,"apples, granny smith, with skin, raw"
3,apple,"apples, honeycrisp, with skin, raw"
4,apple,"apples, raw, gala, with skin (includes foods f..."


### Name matching de-duplication            
            
Allowing many-to-one cases (different food features that can correspond to the same compound, i.e. the same disease relationship), provided that the duplicate term is less than or equal to 5 and those exceeding it are taken to 5;            
not allowing one-to-many cases (same food features corresponding to different disease relationships), with one-to-many treated as one-to-one correspondence            
Cases where many-to-many is not allowed, food compounds are first de-duplicated and then processed as many-to-one.            
            
De-duplication gives priority to retaining those with higher similarity to the target string!

In [99]:
dupli_long = food_name_match.groupby('food_disease')['long'].agg(['count', 'unique']).reset_index()
dupli_fd = food_name_match.groupby('long')['food_disease'].agg(['count', 'unique']).reset_index()

In [100]:
# step 1:One by one correspondence
unique_twoside = food_name_match[
    (food_name_match['long'].isin(dupli_fd.query('count==1')['long'])) &
    (food_name_match['food_disease'].isin(dupli_long.query('count==1')['food_disease']))]
print("一一对应：", len(set(unique_twoside['food_disease'])), 'vs', len(set(unique_twoside['long'])))

一一对应： 367 vs 367


In [101]:
# step 2: Handling duplicates
food_long = set(food_name_match['long'])-set(unique_twoside['long'])
food_cmp = set(food_name_match['food_disease'])-set(unique_twoside['food_disease'])
long_prepare, cmp_prepare = [], []

print("待处理 营养物质食物：", len(food_long), "化合物食物：", len(food_cmp))
for l in food_long:
    sub_df = food_name_match[(food_name_match['long'].isin(food_long)) & (food_name_match['food_disease'].isin(food_cmp))]
    all_fd = sub_df[sub_df['long']==l].reset_index(drop=True)
    if (len(all_fd)==0) or (l not in food_long):
        # The best match has been removed at step1 and cannot be paired
        continue
    elif len(all_fd)==1:
        # Multiple nutrient foods for one compound food, only top 5 nutrient foods are retained
        f_cmp = all_fd['food_disease'][0]
        all_nur = sub_df[sub_df['food_disease']==f_cmp].reset_index(drop=True)
        all_nur['score'] = all_nur['long'].apply(lambda x: name_similar(x, f_cmp))
        all_nur = all_nur.sort_values('score', ascending=False).reset_index(drop=True)
        sub_long = all_nur['long'][0:3]
        sub_nur = all_nur['food_disease'][0:3]
    else:
        # One nutrient food corresponds to multiple compound foods, keeping only one compound food (including many-to-many)
        all_fd['score'] = all_fd['food_disease'].apply(lambda x: name_similar(x, l))
        all_fd = all_fd.sort_values('score', ascending=False).reset_index(drop=True)
        sub_long = [all_fd['long'][0]]
        sub_nur = [all_fd['food_disease'][0]]
        
    cmp_prepare.extend(sub_nur)
    long_prepare.extend(sub_long)
    assert len(cmp_prepare)==len(long_prepare), l
    food_long = food_long - set(sub_long)
    food_cmp = food_cmp - set(sub_nur)
        
print("剩余 营养物质食物：", len(food_long), "化合物食物：", len(food_cmp))

dupli_deal = pd.DataFrame({'food_disease': cmp_prepare, 'long': long_prepare})
print("关系表新增 营养物质食物：", len(set(dupli_deal['long'])), "化合物食物：", len(set(dupli_deal['food_disease'])))

待处理 营养物质食物： 692 化合物食物： 157
剩余 营养物质食物： 468 化合物食物： 37
关系表新增 营养物质食物： 224 化合物食物： 120


In [102]:
# step 3: Splicing the new relationship table
new_match = pd.concat([unique_twoside, dupli_deal]).reset_index(drop=True)
print("营养物质食物：", len(set(new_match['long'])), "化合物食物：", len(set(new_match['food_disease'])))
new_match['food_id'] = list(map(lambda x: 'food_'+str(x), list(range(len(new_match)))))
new_match.head()

营养物质食物： 591 化合物食物： 487


,food_disease,long,food_id
0,alfalfa,"alfalfa sprouts, raw",food_0
1,almond,"almonds, nfs",food_1
2,chicory roots,"chicory roots, raw",food_2
3,jalapeno pepper,"pepper, raw, nfs",food_3
4,black raisin,raisins,food_4


### Associated nutrients, compounds

In [103]:
food_feature_matched = new_match.merge(food_features, left_on='long', right_on= 'description').drop(columns=['food_disease', 'long', 'description'])
for col in food_feature_matched.columns[1:]:
    tt = food_feature_matched[col]
    if sum(np.isnan(tt)) == len(food_feature_matched):
        food_feature_matched = food_feature_matched.drop(columns=[col])
food_feature_matched.head()

,food_id,"ergosta-5,7-dienol","ergosta-7,22-dienol",25-hydroxycholecalciferol,alanine,"alcohol, ethyl",arginine,ash,aspartic acid,beta-sitostanol,...,vitamin d3 (cholecalciferol),vitamin d4,vitamin e (alpha-tocopherol),"vitamin e, added",vitamin k (dihydrophylloquinone),vitamin k (menaquinone-4),vitamin k (phylloquinone),water,zeaxanthin,"zinc, zn"
0,food_0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.02,0.0,NaN,NaN,30.5,92.82,NaN,0.92
1,food_1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,23.90,0.0,NaN,NaN,0.0,2.41,NaN,3.31
2,food_2,NaN,NaN,NaN,NaN,NaN,NaN,0.89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.00,NaN,0.33
3,food_3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.79,0.0,NaN,NaN,6.5,93.30,NaN,0.17
4,food_4,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.12,0.0,NaN,NaN,3.5,15.46,NaN,0.36


In [104]:
food_compound_matched= new_match.merge(food_compound, left_on='food_disease', right_on= 'food_lower')
food_compound_matched = food_compound_matched[['food_id', 'cmp_lower', 'standard_content', 'scaler']]
food_compound_matched.head()

,food_id,cmp_lower,standard_content,scaler
0,food_0,ash,1166.8,0.025652
1,food_0,carbohydrate,19500,0.202252
2,food_0,fat,6650,0.075373
3,food_0,fatty acids,100,0.010825
4,food_0,fiber (dietary),2450,0.101210


## Save results

In [105]:
print(f"营养物质 x 食物（用于构建特征）：{food_feature_matched.T.shape[0]} x {food_feature_matched.T.shape[1]}")
print(f"食物 x 化合物（用于构建标签）：{len(set(food_compound_matched['food_id']))} x {len(set(food_compound_matched['cmp_lower']))}")

营养物质 x 食物（用于构建特征）：213 x 591
食物 x 化合物（用于构建标签）：591 x 1654


In [1]:
# food_feature_matched.to_csv('/home/mw/project/data/中间表/food_feature_matched.csv', index=False)
# new_match.to_csv('/home/mw/project/data/中间表/foodname_match_relation.csv', index=False)
# food_compound_matched.to_csv('/home/mw/project/data/中间表/food_compound_matched.csv', index=False)

### Competition data set slice and dice

In [52]:
# new_match = pd.read_csv('/home/mw/project/data/中间表/foodname_match_relation.csv')
potential = new_match.copy()
potential['id_num'] = potential['food_id'].apply(lambda x: int(str(x).split('_')[1]))
check_multi = potential.groupby('food_disease')['id_num'].agg({'count', max, min, 'mean'}).reset_index()
check_multi = check_multi[check_multi['count']>1]
check_multi['test_a'] = check_multi['min'].apply(lambda x: 'food_'+str(x))
check_multi['test_b'] = check_multi['max'].apply(lambda x: 'food_'+str(x))
check_multi['train'] = check_multi['mean'].apply(lambda x: 'food_'+str(int(x)))
check_multi['train'] = np.where(check_multi['count']==2, np.nan, check_multi['train'])
print(f"有{len(check_multi)}组食物，疾病关系相同")

有63组食物，疾病关系相同


,food_disease,count,min,max,mean,test_a,test_b,train
9,alaska wild rhubarb,2,549,550,549.5,food_549,food_550,NaN
18,apple,3,559,561,560.0,food_559,food_561,food_560
32,avocado,3,496,498,497.0,food_496,food_498,food_497
38,bean,3,393,395,394.0,food_393,food_395,food_394
46,black radish,3,472,474,473.0,food_472,food_474,food_473


In [55]:
unique_ids = new_match[~new_match['food_disease'].isin(check_multi['food_disease'])].reset_index(drop=True)['food_id']
unique_len=len(unique_ids)
print(f"有{unique_len}组食物，疾病关系相异")

有424组食物，疾病关系相异


In [67]:
test_a = list(check_multi['test_a'])
test_b = list(check_multi['test_b'])
train = list(check_multi['train'].dropna())

rand = random.sample(range(0,unique_len), unique_len)
test_a.extend(unique_ids[rand[0:55]])
test_b.extend(unique_ids[rand[55:110]])
train.extend(unique_ids[rand[110:]])

split_resilt = [train, test_a, test_b]
print("训练集：", len(train),"\nA榜测试集：", len(test_a), "\nB榜测试集", len(test_b))

训练集： 355 
A榜测试集： 118 
B榜测试集 118


In [69]:
# with open('/home/mw/project/data/中间表/split_result.pkl','wb') as fw:
#     pkl.dump(split_resilt,fw)